In [1]:
# Preliminary imports and auxiliary-function definition

from pathlib import Path
from devtools import debug
import tempfile

from fractal_server.app.runner.v2.models import Dataset, WorkflowTask
from fractal_server.app.runner.v2.runner import execute_tasks_v2
from fractal_server.images import SingleImage

from concurrent.futures import ThreadPoolExecutor
from image_data_exist_on_disk import image_data_exist_on_disk

import sys
sys.path.append("../tests/v2/04_runner")
from fractal_tasks_core_mock import TASK_LIST

executor = ThreadPoolExecutor()

{'name': 'create_ome_zarr_compound', 'input_types': {}, 'output_types': {}, 'function_non_parallel': <cyfunction create_ome_zarr at 0x750a401c6810>, 'function_parallel': <cyfunction yokogawa_to_zarr at 0x750a401c7ac0>}
{'name': 'create_ome_zarr_multiplex_compound', 'input_types': {}, 'output_types': {}, 'function_non_parallel': <cyfunction create_ome_zarr_multiplex at 0x750a401c7100>, 'function_parallel': <cyfunction yokogawa_to_zarr at 0x750a401c7ac0>}
{'name': 'MIP_compound', 'input_types': {'3D': True}, 'output_types': {'3D': False}, 'function_non_parallel': <cyfunction new_ome_zarr at 0x750a40205d80>, 'function_parallel': <cyfunction maximum_intensity_projection at 0x750a40206740>}
{'name': 'illumination_correction', 'input_types': {'illumination_correction': False}, 'output_types': {'illumination_correction': True}, 'function_non_parallel': None, 'function_parallel': <cyfunction illumination_correction at 0x750a40204380>}
{'name': 'illumination_correction_compound', 'input_types':

In [2]:
# Create temporary directory for mocked zarrs
tmp_path = Path(tempfile.mkdtemp())
zarr_dir = (tmp_path / "zarr_dir").as_posix().rstrip("/")
print(f"{zarr_dir=}")

zarr_dir='/tmp/tmpt_jph5bx/zarr_dir'


In [3]:
# Run create-ome-zarr (starting from empty Dataset)
dataset = execute_tasks_v2(
    wf_task_list=[
        WorkflowTask(
            task=TASK_LIST["create_ome_zarr_compound"],
            args_non_parallel=dict(image_dir="/tmp/input_images"),
        )
    ],
    dataset=Dataset(zarr_dir=zarr_dir),
    executor=executor,
)

# Images were added, but data do not exist on-disk yet
assert image_data_exist_on_disk(dataset.images)

# Print dataset metadata
debug(dataset);

[create_ome_zarr] START
[create_ome_zarr] image_dir='/tmp/input_images'
[create_ome_zarr] zarr_dir='/tmp/tmpt_jph5bx/zarr_dir'
[create_ome_zarr] zarr_path='/tmp/tmpt_jph5bx/zarr_dir/my_plate.zarr'
[create_ome_zarr] END
[yokogawa_to_zarr] START
[yokogawa_to_zarr] path='/tmp/tmpt_jph5bx/zarr_dir/my_plate.zarr/A/01/0'
[yokogawa_to_zarr] raw_path='/tmp/input_images/A_01_0.tif'
[yokogawa_to_zarr] START
[yokogawa_to_zarr] path='/tmp/tmpt_jph5bx/zarr_dir/my_plate.zarr/A/02/0'
[yokogawa_to_zarr] raw_path='/tmp/input_images/A_02_0.tif'
[yokogawa_to_zarr] END
[yokogawa_to_zarr] END
[image_data_exist_on_disk] /tmp/tmpt_jph5bx/zarr_dir/my_plate.zarr/A/01/0 contains data
[image_data_exist_on_disk] /tmp/tmpt_jph5bx/zarr_dir/my_plate.zarr/A/02/0 contains data
/tmp/ipykernel_93002/61380797.py:17 <module>
    dataset: Dataset(
        id=None,
        history=[
            'create_ome_zarr_compound',
        ],
        zarr_dir='/tmp/tmpt_jph5bx/zarr_dir',
        images=[
            SingleImage(
    

In [4]:
# Run illumination correction, with overwrite_input=True
dataset = execute_tasks_v2(
    wf_task_list=[
        WorkflowTask(
            task=TASK_LIST["illumination_correction"],
            args_parallel=dict(overwrite_input=True),
        )
    ],
    dataset=dataset,
    executor=executor,
)

# Image data still exist (and they actually were modified)
assert image_data_exist_on_disk(dataset.images)

# Print dataset metadata
debug(dataset);

[image_data_exist_on_disk] /tmp/tmpt_jph5bx/zarr_dir/my_plate.zarr/A/01/0 contains data
[image_data_exist_on_disk] /tmp/tmpt_jph5bx/zarr_dir/my_plate.zarr/A/02/0 contains data
/tmp/ipykernel_93002/4204713964.py:17 <module>
    dataset: Dataset(
        id=None,
        history=[
            'create_ome_zarr_compound',
            'illumination_correction',
        ],
        zarr_dir='/tmp/tmpt_jph5bx/zarr_dir',
        images=[
            SingleImage(
                path='/tmp/tmpt_jph5bx/zarr_dir/my_plate.zarr/A/01/0',
                origin=None,
                attributes={
                    'well': 'A01',
                    'plate': 'my_plate.zarr',
                },
                types={
                    '3D': True,
                    'illumination_correction': True,
                },
            ),
            SingleImage(
                path='/tmp/tmpt_jph5bx/zarr_dir/my_plate.zarr/A/02/0',
                origin=None,
                attributes={
                

In [5]:
# Run new-ome-zarr, as an init task for MIP
dataset = execute_tasks_v2(
    wf_task_list=[
        WorkflowTask(
            task=TASK_LIST["MIP_compound"],
            args_non_parallel=dict(suffix="mip"),
        )
    ],
    dataset=dataset,
    executor=executor,
)

# 3D image data exist on disk
images_3D = [
        image
        for image in dataset.images
        if image.types["3D"]
    ]
assert image_data_exist_on_disk(images_3D)

# 2D image data do not exist on disk
images_2D = [
        image
        for image in dataset.images
        if not image.types["3D"]
    ]
assert image_data_exist_on_disk(images_2D)

# Print dataset metadata
debug(dataset);

[image_data_exist_on_disk] /tmp/tmpt_jph5bx/zarr_dir/my_plate.zarr/A/01/0 contains data
[image_data_exist_on_disk] /tmp/tmpt_jph5bx/zarr_dir/my_plate.zarr/A/02/0 contains data
[image_data_exist_on_disk] /tmp/tmpt_jph5bx/zarr_dir/my_plate_mip.zarr/A/01/0 contains data
[image_data_exist_on_disk] /tmp/tmpt_jph5bx/zarr_dir/my_plate_mip.zarr/A/02/0 contains data
/tmp/ipykernel_93002/4261258771.py:30 <module>
    dataset: Dataset(
        id=None,
        history=[
            'create_ome_zarr_compound',
            'illumination_correction',
            'MIP_compound',
        ],
        zarr_dir='/tmp/tmpt_jph5bx/zarr_dir',
        images=[
            SingleImage(
                path='/tmp/tmpt_jph5bx/zarr_dir/my_plate.zarr/A/01/0',
                origin=None,
                attributes={
                    'well': 'A01',
                    'plate': 'my_plate.zarr',
                },
                types={
                    '3D': True,
                    'illumination_correction'

In [6]:
# Run cellpose on current images
dataset = execute_tasks_v2(
    wf_task_list=[
        WorkflowTask(
            task=TASK_LIST["cellpose_segmentation"],
            args_parallel=dict(),
        )
    ],
    dataset=dataset,
    executor=executor,
)

# All image data (3D and 2D) now exists on disk
assert image_data_exist_on_disk(dataset.images)

# Print dataset metadata
debug(dataset);

[image_data_exist_on_disk] /tmp/tmpt_jph5bx/zarr_dir/my_plate.zarr/A/01/0 contains data
[image_data_exist_on_disk] /tmp/tmpt_jph5bx/zarr_dir/my_plate.zarr/A/02/0 contains data
[image_data_exist_on_disk] /tmp/tmpt_jph5bx/zarr_dir/my_plate_mip.zarr/A/01/0 contains data
[image_data_exist_on_disk] /tmp/tmpt_jph5bx/zarr_dir/my_plate_mip.zarr/A/02/0 contains data
/tmp/ipykernel_93002/1519895496.py:17 <module>
    dataset: Dataset(
        id=None,
        history=[
            'create_ome_zarr_compound',
            'illumination_correction',
            'MIP_compound',
            'cellpose_segmentation',
        ],
        zarr_dir='/tmp/tmpt_jph5bx/zarr_dir',
        images=[
            SingleImage(
                path='/tmp/tmpt_jph5bx/zarr_dir/my_plate.zarr/A/01/0',
                origin=None,
                attributes={
                    'well': 'A01',
                    'plate': 'my_plate.zarr',
                },
                types={
                    '3D': True,
        

In [7]:
executor.shutdown()